### Описание задачи
Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.

### Задание
Задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

### Входные данные

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

### Важно:

Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
Если товар не встречался в обучающей выборке, его популярность равна 0.
Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.

### Задание

1. На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)
2. Реализуйте два алгоритма рекомендаций:
    - сортировка просмотренных id по популярности (частота появления в просмотренных),
    - сортировка просмотренных id по покупаемости (частота появления в покупках).
3. Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)


 

In [1]:
import pandas as pd
from collections import defaultdict

In [2]:
def read_dataset(fname):
    data = pd.read_csv(fname, sep=';', names=['views', 'purchases'])
    for c in data.columns:
        data[c] = data[c].apply(
            lambda x: [int(v) for v in x.split(',')] if pd.notna(x) else x)
    return data


folder = 'https://raw.githubusercontent.com/chekhovana/courses/main/machine_learning/5_tasks/4_recommender_systems/data/'
train = read_dataset(folder + '4.3_coursera_sessions_train.txt')
test = read_dataset(folder + '4.3_coursera_sessions_test.txt')


In [3]:
def get_counts(x):
    x = x.dropna()
    counter = defaultdict(int)
    for lst in x:
        for val in lst:
            counter[val] += 1
    return counter


def remove_dups(lst):
    seen = set()
    return [x for x in lst if not (x in seen or seen.add(x))]


def sort_by_counts(views, counts):
    return sorted(views, key=lambda x: counts[x], reverse=True)

In [7]:
def get_precision(row, k):
    recommended = row['views'][:k]
    purchased = row['purchases']
    precision = [item for item in purchased if item in recommended]
    return len(precision) / k


def get_recall(row, k):
    recommended = row['views'][:k]
    purchased = row['purchases']
    recall = [item for item in recommended if item in purchased]
    return len(recall) / len(purchased)


def get_metrics(df, counts):
    metrics = df[df['purchases'].notna()].copy()
    metrics['views'] = metrics['views'].apply(remove_dups)
    metrics['views'] = metrics['views'].apply(sort_by_counts, counts=counts)
    metrics = [[round(metrics.apply(f, k=k, axis=1).mean(), 4) 
        for f in (get_recall, get_precision)] for k in [1, 5]]
    return sum(metrics, [])

view_counts = get_counts(train['views'])
purchase_counts = get_counts(train['purchases'])
print('Рекомендации по частоте просмотров, качество на обучающей выборке')
print(get_metrics(train, view_counts))
print('Рекомендации по частоте просмотров, качество на тестовой выборке')
print(get_metrics(train, purchase_counts))
print('Рекомендации по частоте покупок, качество на обучающей выборке')
print(get_metrics(test, view_counts))
print('Рекомендации по частоте покупок, качество на тестовой выборке')
print(get_metrics(test, purchase_counts))

Рекомендации по частоте просмотров, качество на обучающей выборке
[0.4426, 0.5122, 0.8247, 0.2125]
Рекомендации по частоте просмотров, качество на тестовой выборке
[0.6884, 0.8038, 0.9263, 0.2525]
Рекомендации по частоте покупок, качество на обучающей выборке
[0.4173, 0.4813, 0.8, 0.2038]
Рекомендации по частоте покупок, качество на тестовой выборке
[0.4606, 0.5277, 0.8202, 0.2101]
